In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops

# Fungsi untuk menentukan label berdasarkan folder tempat gambar berada
def get_label_from_parent_folder(folder_name):
    if 'gabah beras merah' in folder_name.lower():
        return 'gabah_beras_merah'
    elif 'gabah bangkok' in folder_name.lower():
        return 'gabah_bangkok'
    elif 'gabah ir64' in folder_name.lower():
        return 'gabah_ir64'
    else:
        return None  # Kembalikan None jika tidak ada label yang sesuai

def get_quality_from_subfolder(subfolder_name):
    if 'utuh' in subfolder_name.lower():
        return 'utuh'
    elif 'patah' in subfolder_name.lower():
        return 'patah'
    elif 'rusak' in subfolder_name.lower():
        return 'rusak'
    else:
        return None  # Kembalikan None jika tidak ada label yang sesuai

# Fungsi untuk ekstraksi fitur
def extract_features(image_path, parent_label, quality_label, jenis):
    try:
        # Membaca gambar
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Gambar {image_path} tidak dapat dibaca.")
    except Exception as e:
        print(f"Error pada {image_path}: {e}")
        return None  # Mengembalikan None jika ada error
    
    # Mengubah gambar ke grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Deteksi tepi menggunakan Canny
    edges = cv2.Canny(gray, 100, 200)

    # Mengubah gambar ke ruang warna HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Ekstraksi fitur warna (rata-rata nilai Hue, Saturation, dan Value)
    avg_hue = np.mean(hsv[:, :, 0])  # Rata-rata Hue
    avg_saturation = np.mean(hsv[:, :, 1])  # Rata-rata Saturation
    avg_value = np.mean(hsv[:, :, 2])  # Rata-rata Value
    
    # Ekstraksi fitur tekstur menggunakan GLCM
    glcm = graycomatrix(gray, [1], [0], symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    
    # Menghitung ukuran gambar (pixel)
    height, width = image.shape[:2]
    size = height * width
    
    # Menghitung bentuk menggunakan area kontur
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    area = 0
    for cnt in contours:
        area += cv2.contourArea(cnt)
    
    # Menyusun hasil ekstraksi fitur
    features = {
        'filename': os.path.basename(image_path),
        'avg_hue': avg_hue,  # Hue
        'avg_saturation': avg_saturation,  # Saturation
        'avg_value': avg_value,  # Value
        'contrast': contrast,
        'dissimilarity': dissimilarity,
        'homogeneity': homogeneity,
        'energy': energy,
        'correlation': correlation,
        'size': size,
        'area': area,
        'kind': parent_label,  # Label parent: gabah beras merah, gabah bangkok, ir64
        'quality': quality_label,  # Kualitas gabah: utuh, patah, rusak
    }
    
    return features

# Menyimpan fitur ke dalam DataFrame dan CSV
def save_features_to_csv(input_folder, output_csv):
    data = []
    
    # Iterasi melalui subfolder (gabah beras merah, gabah bangkok, ir64) di dalam folder input
    for parent_folder_name in os.listdir(input_folder):
        parent_folder_path = os.path.join(input_folder, parent_folder_name)
        
        # Pastikan itu adalah folder dan bukan file
        if os.path.isdir(parent_folder_path):
            # Tentukan label parent gabah berdasarkan nama folder
            parent_label = get_label_from_parent_folder(parent_folder_name)
            if parent_label is None:
                print(f"Folder {parent_folder_name} tidak memiliki label yang sesuai, lewati.")
                continue
            
            # Iterasi melalui subfolder (utuh, patah, rusak) di dalam folder parent
            for subfolder_name in os.listdir(parent_folder_path):
                subfolder_path = os.path.join(parent_folder_path, subfolder_name)
                
                # Pastikan itu adalah folder dan bukan file
                if os.path.isdir(subfolder_path):
                    # Tentukan label kualitas gabah berdasarkan nama subfolder
                    quality_label = get_quality_from_subfolder(subfolder_name)
                    if quality_label is None:
                        print(f"Subfolder {subfolder_name} tidak memiliki label yang sesuai, lewati.")
                        continue
                    
                    # Input manual untuk 'jenis' gabah
                    jenis = input(f"Masukkan jenis gabah untuk folder '{subfolder_name}' '{parent_folder_name}': ")

                    # Iterasi melalui file gambar dalam subfolder
                    for filename in os.listdir(subfolder_path):
                        file_path = os.path.join(subfolder_path, filename)
                        
                        # Memeriksa ekstensi file untuk memastikan itu adalah gambar
                        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                            print(f"File {filename} bukan gambar, lewati.")
                            continue
                        
                        # Ekstrak fitur dari gambar
                        features = extract_features(file_path, parent_label, quality_label, jenis)
                        if features:  # Hanya menambahkan fitur yang valid
                            data.append(features)
                        else:
                            print(f"Gambar {filename} gagal diproses.")
    
    # Membuat DataFrame dari data
    df = pd.DataFrame(data)
    
    # Menyimpan DataFrame ke CSV
    df.to_csv(output_csv, index=False)
    print(f"Data berhasil disimpan dalam {output_csv}")

# Direktori gambar (folder utama yang berisi subfolder gabah beras merah, gabah bangkok, ir64)
input_folder = r'C:\Users\Naufal\OneDrive\MyDocuments\KULIAH\Tugas Kuliah\TUGAS AKHIR'  # Ganti dengan direktori gambar Anda
output_csv = 'gabah_features_completed.csv'

# Menyimpan fitur ke dalam CSV
save_features_to_csv(input_folder, output_csv)


Masukkan jenis gabah untuk folder 'patah' 'gabah bangkok':  patah
Masukkan jenis gabah untuk folder 'rusak' 'gabah bangkok':  rusak
Masukkan jenis gabah untuk folder 'utuh' 'gabah bangkok':  utuh
Masukkan jenis gabah untuk folder 'patah' 'gabah beras merah':  patah
Masukkan jenis gabah untuk folder 'rusak' 'gabah beras merah':  rusak
Masukkan jenis gabah untuk folder 'utuh' 'gabah beras merah':  utuh
Masukkan jenis gabah untuk folder 'patah' 'gabah ir64':  patah
Masukkan jenis gabah untuk folder 'rusak' 'gabah ir64':  rusak
Masukkan jenis gabah untuk folder 'utuh' 'gabah ir64':  utuh


Folder referensi tidak memiliki label yang sesuai, lewati.
Data berhasil disimpan dalam gabah_features_completed.csv
